In [1]:
import pymysql
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")


def getSqlResult(sql):
    cursor.execute(sql)
    res_rows = cursor.fetchall()
    df = pd.DataFrame(res_rows)
    return df

INFO:root:connection created


In [2]:
from datetime import datetime

def NFTSoldProfitQuery(start_dt,end_dt):
    sql = """
insert into dm.`dm_nft_sold_profit_eth_di_test` (
    `sell_transaction_hash`,
    `sell_block_number`,
    `sell_log_index`,
    `sell_ts`,
    `account_address`,
    `token_address`,
    `token_id`,
    `erc_standard`,
    `sell_token_num`,
    `sell_eth_amt`,
    `sell_gas_cost`,
    `seller_receive_eth_amount`,
    `is_sell_scalp`,
    `sell_trade_type`,
    `buy_transaction_hash`,
    `buy_block_number`,
    `buy_ts`,
    `buy_token_num`,
    `buyer_pay_eth_amount`,
    `is_buy_scalp`,
    `buy_trade_type`,
    `profit`
)
with nft_sold_drv as (
    select from_address as account_address
        ,token_address
        ,token_ids
        ,token_num
        ,trade_eth_value
        ,gas_cost
        ,trade_time
        ,transaction_hash
        ,log_index
        ,block_number
        ,trade_type
        ,erc_standard
        ,seller_receive_eth_amount
        ,is_scalp
    from dw.dwb_nft_trade_eth_detail_hi
    where trade_time >= '{start_dt}'
    and trade_time <= '{end_dt}'
    and erc_standard not in ('ERC1155')
)
,nft_get_drv as (
    select to_address as account_address
        ,token_address
        ,token_ids
        ,token_num
        ,buyer_pay_eth_amount as cost_eth_value
        ,trade_time as ts
        ,transaction_hash
        ,block_number
        ,trade_type
        ,is_scalp
    from dw.dwb_nft_trade_eth_detail_hi
    where trade_time <= '{end_dt}'
    and erc_standard not in ('ERC1155')
    union all
    select account_address
        ,token_address
        ,token_id as token_ids
        ,mint_amount as token_num
        ,(gas_cost+mint_cost) as cost_eth_value
        ,mint_time as ts
        ,transaction_hash
        ,block_number
        ,'mint' as trade_type
        ,0 as is_scalp
    from dw.dwb_nft_mint_detail_eth_hi
    where mint_time <= '{end_dt}'
)
select
    `sell_transaction_hash`,
    `sell_block_number`,
    `sell_log_index`,
    `sell_ts`,
    `account_address`,
    `token_address`,
    `token_id`,
    `erc_standard`,
    `sell_token_num`,
    `sell_eth_amt`,
    `sell_gas_cost`,
    `seller_receive_eth_amount`,
    `is_sell_scalp`,
    `sell_trade_type`,
    `buy_transaction_hash`,
    `buy_block_number`,
    `buy_ts`,
    `buy_token_num`,
    `buyer_pay_eth_amount`,
    `is_buy_scalp`,
    `buy_trade_type`,
    `profit`
from
(
    select a.account_address
        ,a.token_address
        ,a.token_ids as token_id
        ,a.erc_standard
        ,a.trade_time as sell_ts
        ,a.transaction_hash as sell_transaction_hash
        ,a.block_number as sell_block_number
        ,a.log_index as sell_log_index
        ,a.token_num as sell_token_num
        ,a.trade_eth_value as sell_eth_amt
        ,a.gas_cost as sell_gas_cost
        ,a.seller_receive_eth_amount
        ,a.is_scalp as is_sell_scalp
        ,a.trade_type as sell_trade_type

        ,b.ts as buy_ts
        ,b.token_num as buy_token_num
        ,b.cost_eth_value as buyer_pay_eth_amount
        ,b.transaction_hash as buy_transaction_hash
        ,b.block_number as buy_block_number
        ,b.is_scalp as is_buy_scalp
        ,b.trade_type as buy_trade_type
        ,COALESCE(a.seller_receive_eth_amount,0)-coalesce(b.cost_eth_value,0) as profit
        ,ROW_NUMBER()over(partition by a.transaction_hash,a.log_index,a.block_number,a.trade_time order by b.ts desc) as sold_pre_buy_rnk
    from nft_sold_drv as a
    left join nft_get_drv as b
    on a.token_address = b.token_address
    and a.token_ids = b.token_ids
    and a.trade_time > b.ts
) as drv
where sold_pre_buy_rnk = 1
""".format(
        start_dt = start_dt,
        end_dt = end_dt
    )
    return sql

# start_dt = datetime(2021,12,1).date()
# end_dt = datetime(2021,12,2).date()
# start_str = start_dt.strftime('%Y-%m-%d')
# end_str = end_dt.strftime('%Y-%m-%d')
# print(NFTSoldProfitQuery(start_str,end_str))

In [3]:
from datetime import datetime
from datetime import timedelta

def backfill_data(start_date,end_date):
    end_dt = end_date
    while end_dt > datetime(2017,1,1).date() and end_dt >= start_date:
        if end_dt > datetime(2022,1,1).date():
            start_dt = end_dt - timedelta(days=15)
        elif end_dt > datetime(2021,12,1).date():
            start_dt = end_dt - timedelta(days=3)
        elif end_dt > datetime(2020,6,1).date():
            start_dt = end_dt - timedelta(days=30)
        else:
            start_dt = end_dt - timedelta(days=180)
        print(start_dt,end_dt)
        start_str = start_dt.strftime('%Y-%m-%d')
        end_str = end_dt.strftime('%Y-%m-%d')
        sql = NFTSoldProfitQuery(start_str,end_str)
        cursor.execute(sql)
        end_dt = start_dt
    
    start_dt = start_date
    print(start_dt,end_dt)
    start_str = start_dt.strftime('%Y-%m-%d')
    end_str = end_dt.strftime('%Y-%m-%d')
    sql = NFTSoldProfitQuery(start_str,end_str)
    cursor.execute(sql)
    return True

end_date = datetime.now().date()
# end_dt = datetime(2022,12,17).date()
start_date = datetime(2016,1,1).date()
backfill_data(start_date,end_date)

2023-01-03 2023-01-18
2022-12-19 2023-01-03
2022-12-04 2022-12-19
2022-11-19 2022-12-04
2022-11-04 2022-11-19
2022-10-20 2022-11-04
2022-10-05 2022-10-20
2022-09-20 2022-10-05
2022-09-05 2022-09-20
2022-08-21 2022-09-05
2022-08-06 2022-08-21
2022-07-22 2022-08-06
2022-07-07 2022-07-22
2022-06-22 2022-07-07
2022-06-07 2022-06-22
2022-05-23 2022-06-07
2022-05-08 2022-05-23
2022-04-23 2022-05-08
2022-04-08 2022-04-23
2022-03-24 2022-04-08
2022-03-09 2022-03-24
2022-02-22 2022-03-09
2022-02-07 2022-02-22
2022-01-23 2022-02-07
2022-01-08 2022-01-23
2021-12-24 2022-01-08
2021-12-21 2021-12-24
2021-12-18 2021-12-21
2021-12-15 2021-12-18
2021-12-12 2021-12-15
2021-12-09 2021-12-12
2021-12-06 2021-12-09
2021-12-03 2021-12-06
2021-11-30 2021-12-03
2021-10-31 2021-11-30
2021-10-01 2021-10-31
2021-09-01 2021-10-01
2021-08-02 2021-09-01
2021-07-03 2021-08-02
2021-06-03 2021-07-03
2021-05-04 2021-06-03
2021-04-04 2021-05-04
2021-03-05 2021-04-04
2021-02-03 2021-03-05
2021-01-04 2021-02-03
2020-12-05

True

In [ ]:

DROP TABLE IF EXISTS dm.`dm_nft_sold_profit_eth_di`;
CREATE TABLE dm.`dm_nft_sold_profit_eth_di` (
  `sell_transaction_hash` varchar(80) NOT NULL COMMENT "nft卖出transaction hash",
  `sell_block_number` bigint(20) NOT NULL COMMENT "nft卖出交易的block_number",
  `sell_log_index` int(11) NOT NULL COMMENT "nft卖出交易的log_index",
  `sell_ts` datetime NOT NULL COMMENT "nft卖出时间",
  `account_address` varchar(42) NOT NULL COMMENT "the account_address sold the nft",
  `token_address` varchar(65533) NOT NULL COMMENT "the nft token_address been sold",
  `token_id` varchar(65533) NULL COMMENT "the nft token_id been sold",
  `erc_standard` varchar(20) NULL COMMENT "",
  `sell_token_num` bigint(20) NULL COMMENT "卖出nft数量",
  `sell_eth_amt` double NULL COMMENT "该笔卖出交易eth价值",
  `sell_gas_cost` double NULL COMMENT "该笔卖出交易花费Gas /eth",
  `seller_receive_eth_amount` double NULL COMMENT "该笔卖出交易 卖家收入的金额",
    `is_sell_scalp` INT(10) NULL COMMENT "卖出单是否是刷单",
	`sell_trade_type` varchar(80) NULL COMMENT "卖出交易类型：single trade/bundle trade",
	`buy_transaction_hash` varchar(80) NULL COMMENT "卖出之前的一笔买入的 transaction hash",
	`buy_block_number` varchar(80) NULL COMMENT "nft卖出交易的block_number",
  `buy_ts` varchar(255) NULL COMMENT "卖出之前的一笔买入的 买入时间",
  `buy_token_num` bigint(20) NULL COMMENT "卖出之前的一笔买入的 nft数量",
	`buyer_pay_eth_amount` double NULL COMMENT "该笔卖出交易对应的买入交易 买家支付的金额",
	`is_buy_scalp` INT(10) NULL COMMENT "买入单是否是刷单",
	`buy_trade_type` varchar(80) NULL COMMENT "买入交易类型：mint/single trade/bundle trade",
  `profit` double NULL COMMENT "盈利"
) ENGINE=OLAP
UNIQUE KEY(`sell_transaction_hash`, `sell_block_number`, `sell_log_index`, `sell_ts`)
COMMENT "OLAP"
PARTITION BY RANGE(`sell_ts`)
(PARTITION p201601 VALUES [('2016-01-01 00:00:00'), ('2017-01-01 00:00:00')),
PARTITION p201701 VALUES [('2017-01-01 00:00:00'), ('2018-01-01 00:00:00')),
PARTITION p201801 VALUES [('2018-01-01 00:00:00'), ('2019-01-01 00:00:00')),
PARTITION p201901 VALUES [('2019-01-01 00:00:00'), ('2020-01-01 00:00:00')),
PARTITION p202001 VALUES [('2020-01-01 00:00:00'), ('2021-01-01 00:00:00')),
PARTITION p202101 VALUES [('2021-01-01 00:00:00'), ('2021-03-01 00:00:00')),
PARTITION p202103 VALUES [('2021-03-01 00:00:00'), ('2021-05-01 00:00:00')),
PARTITION p202105 VALUES [('2021-05-01 00:00:00'), ('2021-07-01 00:00:00')),
PARTITION p202107 VALUES [('2021-07-01 00:00:00'), ('2021-09-01 00:00:00')),
PARTITION p202109 VALUES [('2021-09-01 00:00:00'), ('2021-11-01 00:00:00')),
PARTITION p2021011 VALUES [('2021-11-01 00:00:00'), ('2022-01-01 00:00:00')),
PARTITION p202201 VALUES [('2022-01-01 00:00:00'), ('2022-02-01 00:00:00')),
PARTITION p202202 VALUES [('2022-02-01 00:00:00'), ('2022-03-01 00:00:00')),
PARTITION p202203 VALUES [('2022-03-01 00:00:00'), ('2022-04-01 00:00:00')),
PARTITION p202204 VALUES [('2022-04-01 00:00:00'), ('2022-05-01 00:00:00')),
PARTITION p202205 VALUES [('2022-05-01 00:00:00'), ('2022-06-01 00:00:00')),
PARTITION p202206 VALUES [('2022-06-01 00:00:00'), ('2022-07-01 00:00:00')),
PARTITION p202207 VALUES [('2022-07-01 00:00:00'), ('2022-08-01 00:00:00')),
PARTITION p202208 VALUES [('2022-08-01 00:00:00'), ('2022-09-01 00:00:00')),
PARTITION p202209 VALUES [('2022-09-01 00:00:00'), ('2022-10-01 00:00:00')),
PARTITION p202210 VALUES [('2022-10-01 00:00:00'), ('2022-11-01 00:00:00')),
PARTITION p202211 VALUES [('2022-11-01 00:00:00'), ('2022-12-01 00:00:00')),
PARTITION p202212 VALUES [('2022-12-01 00:00:00'), ('2023-01-01 00:00:00')),
PARTITION p202301 VALUES [('2023-01-01 00:00:00'), ('2023-02-01 00:00:00')))
DISTRIBUTED BY HASH(`sell_transaction_hash`) BUCKETS 15
PROPERTIES (
"replication_num" = "3",
"dynamic_partition.enable" = "true",
"dynamic_partition.time_unit" = "MONTH",
"dynamic_partition.time_zone" = "Etc/UTC",
"dynamic_partition.start" = "-2147483648",
"dynamic_partition.end" = "2",
"dynamic_partition.prefix" = "p",
"dynamic_partition.buckets" = "32",
"dynamic_partition.replication_num" = "3",
"dynamic_partition.start_day_of_month" = "1",
"in_memory" = "false",
"storage_format" = "V2"
);

In [ ]:
ALTER TABLE `dm_nft_sold_profit_eth_hi_test` SWAP WITH `dm_nft_sold_profit_eth_hi`;